<a href="https://colab.research.google.com/github/EstebanAG1005/Laboratorio7-DL/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AlexNet para CIFAR10

# Importando las bibliotecas necesarias
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim


In [ ]:
# Configurando el dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# Data augmentation for training set
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [ ]:
# Descargando y cargando el dataset CIFAR10
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

100%|██████████| 170498071/170498071 [00:01<00:00, 105139303.14it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Definición de AlexNet
class ModifiedAlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedAlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 8 * 8, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [ ]:
model = ModifiedAlexNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# Training for more epochs
num_epochs = 75
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Adjust learning rate based on the loss
    scheduler.step(loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/75], Loss: 1.4184
Epoch [2/75], Loss: 1.3427
Epoch [3/75], Loss: 1.2955
Epoch [4/75], Loss: 1.0673
Epoch [5/75], Loss: 1.2660
Epoch [6/75], Loss: 1.0321
Epoch [7/75], Loss: 1.0029
Epoch [8/75], Loss: 0.7912
Epoch [9/75], Loss: 1.0037
Epoch [10/75], Loss: 0.7044
Epoch [11/75], Loss: 0.7708
Epoch [12/75], Loss: 0.7716
Epoch [13/75], Loss: 0.4997
Epoch [14/75], Loss: 0.4882
Epoch [15/75], Loss: 0.7219
Epoch [16/75], Loss: 0.4468
Epoch [17/75], Loss: 0.4569
Epoch [18/75], Loss: 0.4179
Epoch [19/75], Loss: 0.4956
Epoch [20/75], Loss: 0.4543
Epoch [21/75], Loss: 0.3765
Epoch [22/75], Loss: 0.3514
Epoch [23/75], Loss: 0.4599
Epoch [24/75], Loss: 0.4184
Epoch [25/75], Loss: 0.2786
Epoch [26/75], Loss: 0.4023
Epoch [27/75], Loss: 0.4409
Epoch [28/75], Loss: 0.2982
Epoch [29/75], Loss: 0.4064
Epoch [30/75], Loss: 0.2162
Epoch [31/75], Loss: 0.1808
Epoch [32/75], Loss: 0.2266
Epoch [33/75], Loss: 0.1724
Epoch [34/75], Loss: 0.3045
Epoch [35/75], Loss: 0.3686
Epoch [36/75], Loss: 0.0977
E

In [ ]:
# Métrica de desempeño: Precisión (Accuracy)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total} %')

Accuracy of the model on the 10000 test images: 81.27 %
